In [ ]:
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib as mpl

import cv2
import os
import pickle as pic
from collections import deque
import sys

loss_function = tf.keras.losses.Huber()
initializer = tf.keras.initializers.VarianceScaling(scale=2.0)

In [ ]:
!nvidia-smi

In [ ]:
from psutil import *

cpu_count()

In [ ]:
!cat /proc/cpuinfo

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
os.chdir("drive/My Drive/colabpro_drive")

In [ ]:
pwd

In [ ]:
!apt update
!apt install xvfb

In [ ]:
nb_path = "/content/drive/My Drive/colabpro_drive/lib"

sys.path.append(nb_path)

# !pip install pyvirtualdisplay --target="{nb_path}" --upgrade 

import pyvirtualdisplay
from gym.wrappers import Monitor
import glob
import io
import base64
from IPython import display as ipythondisplay
from IPython.display import HTML

In [ ]:
d = pyvirtualdisplay.Display()
d.start()

In [ ]:
def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay loop controls style="height: 400px;">
      <source src="data:video/mp4;base64,{0}" type="video/mp4" />
      </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

In [ ]:
import gym
import cv2
import numpy as np

import plotly.graph_objs as go
import pickle as p

from sklearn.manifold import TSNE

import pandas as pd
import plotly.express as px

import tensorflow as tf
load_model = tf.keras.models.load_model

In [ ]:
with open('reward_history', 'rb') as file:
    history = p.load(file)

In [ ]:
h1 = go.Scatter(y=history, 
                    mode="lines", line=dict(
                    width=2,
                    color='blue'),
                    name="reward"
                   )

data = [h1]
layout1 = go.Layout(title='Total Reward',
                   xaxis=dict(title='Episode'),
                   yaxis=dict(title=''))
fig1 = go.Figure(data = data, layout=layout1)
fig1.show(renderer="colab")

In [ ]:
def screen_pixel_preprocess(observation):
    s = cv2.cvtColor(observation, cv2.COLOR_BGR2GRAY)
    s = cv2.resize(s, (0, 0), fx=0.5, fy=0.5, interpolation = cv2.INTER_AREA) 
    # cv2.imwrite('image.png',s) 
    s = s/236.0
    return s

def update_state(state,observation):
    ds_observation = screen_pixel_preprocess(observation)
    state.append(ds_observation)
    if len(state) > 4:
        del state[:1]

def predict_action(model, s):
    return np.argmax(model.predict(np.array([np.stack((s[0],s[1],s[2],s[3]),axis=2)]))[0])

In [ ]:
model = load_model('model/model-700.h5')
model.summary()

In [ ]:
!wget http://www.atarimania.com/roms/Roms.rar
!mkdir ROM
!unrar e Roms.rar ROM
!python -m atari_py.import_roms ROM

In [ ]:
env = wrap_env(gym.make('Pong-v0'))
observation = env.reset()
state = []
update_state(state,observation)

while True:
    env.render()
    if len(state) < 4:
        action = env.action_space.sample()
    else:
        action = predict_action(model, state)
    
    observation, reward, done, _ = env.step(action)

    update_state(state,observation)
    if done: 
        break

env.close()
show_video()

# **Show the State**

In [ ]:
intermediate_model = tf.keras.models.Model(inputs=model.inputs, outputs=model.layers[3].output)
intermediate_model.summary()

In [ ]:
def predict_action(model, intermediate_model, s):
    vector = intermediate_model.predict(np.array([np.stack((s[0],s[1],s[2],s[3]),axis=2)]))[0]
    return np.argmax(model.predict(np.array([np.stack((s[0],s[1],s[2],s[3]),axis=2)]))[0]), vector

In [ ]:
env = wrap_env(gym.make('Pong-v0'))

feature_vector = []
y = []
point = 0
num_frame = 0

observation = env.reset()
state = []
update_state(state,observation)

while True:
    if len(state) < 4:
        action = env.action_space.sample()
    else:
        action, vector = predict_action(model, intermediate_model, state)
        feature_vector.append(vector)
        y.append(point) 

        num_frame+=1

    observation, reward, done, _ = env.step(action)
    if reward != 0:
        point+=1

        for i in range(len(y) - num_frame, len(y)):
            y[i] = y[i]*reward
            
        num_frame = 0
    update_state(state,observation)
    
    if done: 
        break
            
env.close()
show_video()

In [ ]:
len(feature_vector), feature_vector[0].shape

In [ ]:
tsne = TSNE(random_state = 99, n_components=2,verbose=1, n_iter=3000).fit_transform(feature_vector)
tsne.shape

In [ ]:
tsne_pd = pd.DataFrame(tsne, columns=['x', 'y'])
y_pd = pd.DataFrame(y, columns=['point'])

df = pd.concat([tsne_pd, y_pd], axis=1)
df["point"] = df["point"].astype(str)

fig = px.scatter(df, x="x", y="y", color="point")
fig.update_layout(autosize=False, width=1200, height=600)